In [132]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import pandas_profiling 
import matplotlib.pyplot as plt
import sklearn 
import xgboost as xgb
import pickle
from sklearn.preprocessing import LabelEncoder
from scipy import stats
%matplotlib inline
sns.set_style('darkgrid')
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn import model_selection
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc
from pandas_profiling import ProfileReport
from sklearn.preprocessing import PolynomialFeatures
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering 
from scipy.cluster.hierarchy import cophenet, dendrogram, linkage
from scipy.spatial.distance import pdist  
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings("ignore")

# Importing SVD 
from scipy.sparse.linalg import svds

# Importing SVD, KNN with Means, cross validate from surprise
from surprise import SVD, Dataset, Reader, KNNWithMeans
from surprise.model_selection import cross_validate, train_test_split
from surprise import accuracy
from collections import defaultdict


# <span style="font-family: Arial; font-weight:bold;font-size:1.2em;color:#da614e"> PART-A

**DOMAIN**: Smartphone, Electronics

**CONTEXT**: India is the second largest market globally for smartphones after China. About 134 million smartphones were sold across India
in the year 2017 and is estimated to increase to about 442 million in 2022. India ranked second in the average time spent on mobile web by
smartphone users across Asia Pacific. The combination of very high sales volumes and the average smartphone consumer behaviour has
made India a very attractive market for foreign vendors. As per Consumer behaviour, 97% of consumers turn to a search engine when they
are buying a product vs. 15% who turn to social media. If a seller succeeds to publish smartphones based on user’s behaviour/choice at the
right place, there are 90% chances that user will enquire for the same. This Case Study is targeted to build a recommendation system
based on individual consumer’s behaviour or choice.

**DATA DESCRIPTION**:
- author : name of the person who gave the rating
- country : country the person who gave the rating belongs to
- data : date of the rating
- domain: website from which the rating was taken from
- extract: rating content
- language: language in which the rating was given
- product: name of the product/mobile phone for which the rating was given
- score: average rating for the phone
- score_max: highest rating given for the phone
- source: source from where the rating was taken

**PROJECT OBJECTIVE**: We will build a recommendation system using popularity based and collaborative iltering methods to recommend
mobile phones to a user which are most popular and personalised respectively.

## <span style="font-family: Arial; font-weight:bold;font-size:1.2em;color:#8B1A1A"> 1. Import the necessary libraries and read the provided CSVs as a data frame and perform the below steps.

<span style="font-family: Arial; font-weight:bold;font-size:1.1em;color:#0e92ea"> 1.A Merge all the provided CSVs into one dataFrame.

In [2]:
phone_user_review1_df=pd.read_csv('Data Set/Data Set/phone_user_review_file_1.csv',  encoding='latin-1')
phone_user_review2_df=pd.read_csv('Data Set/Data Set/phone_user_review_file_2.csv', encoding='latin-1')
phone_user_review3_df=pd.read_csv('Data Set/Data Set/phone_user_review_file_3.csv', encoding='latin-1')
phone_user_review4_df=pd.read_csv('Data Set/Data Set/phone_user_review_file_4.csv', encoding='latin-1')
phone_user_review5_df=pd.read_csv('Data Set/Data Set/phone_user_review_file_5.csv', encoding='latin-1')
phone_user_review6_df=pd.read_csv('Data Set/Data Set/phone_user_review_file_6.csv', encoding='latin-1')

In [3]:
# Concatenating dataframes without duplicates 
combined_phone_user_review_df = pd.concat([phone_user_review1_df
                                           ,phone_user_review2_df, phone_user_review3_df, 
                                           phone_user_review4_df, phone_user_review5_df,phone_user_review6_df
                                          ],ignore_index=True, sort=True)

In [4]:
combined_phone_user_review_df.sample(5)

,author,country,date,domain,extract,lang,phone_url,product,score,score_max,source
232752,Peter J.,us,10/26/2015,newegg.com,I've ordered a few different cases for it and ...,en,/cellphones/asus-zenfone-2-ze551ml/,"ASUS Zenfone 2 Unlocked Smart Phone, 5.5"" Silv...",10.0,10.0,Newegg
1111049,Charles Akpan,de,12/11/2014,amazon.de,Nie wieder wÃ¼rde ich mein hart verdientes Gel...,de,/cellphones/sony-ericsson-txt/,"Sony Ericsson txt Smartphone (6,3 cm (2,5 Zoll...",6.0,10.0,Amazon
308553,Probal Acharyya,in,4/11/2016,amazon.in,"I got this product on the GREAT INDIAN SALE, 9...",en,/cellphones/asus-zenfone-max-zc550kl/,"Asus Zenfone Max ZC550KL-6A068IN (Black, 2GB, ...",2.0,10.0,Amazon
1065391,Raffaellka,ru,11/20/2010,irecommend.ru,NaN,ru,/cellphones/samsung-monte-s5620/,Samsung GT-S5620 Monte,10.0,10.0,Irecommend
988141,Shawnna Smyrson,us,12/1/2010,amazon.com,The phone was under contract under someone els...,en,/cellphones/apple-iphone-4/,"Apple iPhone 4 Verizon Cellphone, 8GB, White",2.0,10.0,Amazon


<span style="font-family: Arial; font-weight:bold;font-size:1.1em;color:#0e92ea"> 1.B Explore, understand the Data and share at least 2 observations.

In [5]:
combined_phone_user_review_df.describe().T

,count,mean,std,min,25%,50%,75%,max
score,1351644.0,8.00706,2.616121,0.2,7.2,9.2,10.0,10.0
score_max,1351644.0,10.00000,0.000000,10.0,10.0,10.0,10.0,10.0


In [6]:
combined_phone_user_review_df.shape

(1415133, 11)

In [7]:
#Checking for percentage of missing values in columns
for column in combined_phone_user_review_df.columns:
    val = combined_phone_user_review_df[column].isnull().sum()/len(combined_phone_user_review_df[column])
    print(f"The percentage of null values in {column}: {val} %")

The percentage of null values in author: 0.04466152651376231 %
The percentage of null values in country: 0.0 %
The percentage of null values in date: 0.0 %
The percentage of null values in domain: 0.0 %
The percentage of null values in extract: 0.013681399557497422 %
The percentage of null values in lang: 0.0 %
The percentage of null values in phone_url: 0.0 %
The percentage of null values in product: 7.066473610607624e-07 %
The percentage of null values in score: 0.04486433430638675 %
The percentage of null values in score_max: 0.04486433430638675 %
The percentage of null values in source: 0.0 %


In [8]:
combined_phone_user_review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1415133 entries, 0 to 1415132
Data columns (total 11 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   author     1351931 non-null  object 
 1   country    1415133 non-null  object 
 2   date       1415133 non-null  object 
 3   domain     1415133 non-null  object 
 4   extract    1395772 non-null  object 
 5   lang       1415133 non-null  object 
 6   phone_url  1415133 non-null  object 
 7   product    1415132 non-null  object 
 8   score      1351644 non-null  float64
 9   score_max  1351644 non-null  float64
 10  source     1415133 non-null  object 
dtypes: float64(2), object(9)
memory usage: 118.8+ MB


#### Observation: 
<ol>
<li><span style="color:#228B22"> The data has 1415133 rows x 11 columns.
<li><span style="color:#228B22"> Columns score and score_max are float64 data type, other features are object.
<li><span style="color:#228B22"> Some columns have missing values.

<span style="font-family: Arial; font-weight:bold;font-size:1.1em;color:#0e92ea"> 1.C Round fo scores to the nearest integers.

In [9]:
combined_phone_user_review_df['score'] = combined_phone_user_review_df['score'].round(0).astype('float64')

In [10]:
combined_phone_user_review_df['score'].sample(5)

531835     10.0
1294635     8.0
73583      10.0
92875       8.0
1308137     NaN
Name: score, dtype: float64

<span style="font-family: Arial; font-weight:bold;font-size:1.1em;color:#0e92ea"> 1.D Check for missing values. Impute the missing values, if any.

In [11]:
combined_phone_user_review_df.isnull().sum()

author       63202
country          0
date             0
domain           0
extract      19361
lang             0
phone_url        0
product          1
score        63489
score_max    63489
source           0
dtype: int64

In [12]:
#Since the percentage of null values is very less we drop those 
combined_phone_user_review_df.dropna(inplace=True)

<span style="font-family: Arial; font-weight:bold;font-size:1.1em;color:#0e92ea"> 1.E Check for duplicate values and remove them, if any.

In [13]:
#Check for duplicates values
combined_phone_user_review_df.duplicated().any()

True

In [14]:
#Dropping Duplicate records in the Dataset

combined_phone_user_review_df.drop_duplicates(keep='first',inplace=True)

In [15]:
# Verifying absence of Duplicate Records in the Dataset again

combined_phone_user_review_df.duplicated().sum()

0

In [16]:
combined_phone_user_review_df.shape

(1271437, 11)

<span style="font-family: Arial; font-weight:bold;font-size:1.1em;color:#0e92ea"> 1.F Keep only 1 Million data samples. Use random state=612.

In [78]:
# Limiting Dataset to 1 Million Records

df_sample = combined_phone_user_review_df.sample(n=1000000,random_state=612)

In [18]:
df_sample.shape

(1000000, 11)

<span style="font-family: Arial; font-weight:bold;font-size:1.1em;color:#0e92ea"> 1.G. Drop irrelevant features. Keep features like Author, Product, and Score.

In [85]:
#Dropping irrelevant features and retaining relevant features
df_sample.drop(df_sample.columns.difference(['author','product','score']),inplace=True,axis=1)

In [86]:
df_sample.head()

,author,product,score
510111,KHILESH KUMAR VERMA,"Lenovo Vibe K5 (Gold, VoLTE update)",10.0
104794,Evyta,Samsung Galaxy S6,8.0
1222473,VanRaZor,Sony Ericsson K810i,8.0
452910,ruga,Sony Xperia Z2 (Black),6.0
16933,einer Kundin,"Samsung Galaxy S7 edge Smartphone, 13,9 cm (5,...",10.0


## <span style="font-family: Arial; font-weight:bold;font-size:1.2em;color:#8B1A1A"> 2.Answer the following questions

<span style="font-family: Arial; font-weight:bold;font-size:1.1em;color:#0e92ea"> 2.A Identify the most rated products.

In [21]:
# The most rated Products:

rating_per_prod = df_sample.groupby(by='product')['score'].count().sort_values(ascending=False)
rating_per_prod.head(5)

product
Lenovo Vibe K4 Note (White,16GB)     4109
Lenovo Vibe K4 Note (Black, 16GB)    3451
OnePlus 3 (Graphite, 64 GB)          3212
OnePlus 3 (Soft Gold, 64 GB)         2798
Huawei P8lite zwart / 16 GB          2121
Name: score, dtype: int64

<span style="font-family: Arial; font-weight:bold;font-size:1.1em;color:#0e92ea"> 2.B Identify the users with most number of reviews.

In [22]:
# Users with most number of reviews

rating_per_user = df_sample.groupby(by='author')['score'].count().sort_values(ascending=False)
rating_per_user.head(5)

author
Amazon Customer    60408
Cliente Amazon     15051
e-bit               6651
Client d'Amazon     6087
Amazon Kunde        3683
Name: score, dtype: int64

<span style="font-family: Arial; font-weight:bold;font-size:1.1em;color:#0e92ea"> 2.C Select the data with products having more than 50 ratings and users who have given more than 50 ratings. Report the shape of the final dataset.

In [87]:
#Products with more than 50 ratings and users with more than 50 ratings
top_author = df_sample['author'].value_counts()>50
# top_prods = df_sample[df_sample['author'].isin(Count_author[Count_author > 50].index)]
top_author = top_author[top_author].index.tolist()
top_prod = df_sample['product'].value_counts()>50
# top_users = df_sample[df_sample['product'].isin(Count_prod[Count_prod > 50].index)]
top_prod = top_prod[top_prod].index.tolist()

In [89]:
# df_top = pd.merge(top_users,top_prods,how='inner')
df_top = df_sample[(df_sample['product'].isin(top_prod)) & (df_sample['author'].isin(top_author))]

In [90]:
#shape of the dataframe with top 50 ratings of products and users
print(f'Shape of the new dataset with top ratings of top users: {df_top.shape}')

Shape of the new dataset with top ratings of top users: (111595, 3)


In [91]:
df_top

,author,product,score
16933,einer Kundin,"Samsung Galaxy S7 edge Smartphone, 13,9 cm (5,...",10.0
875609,Cliente Amazon,"Samsung E1200 Telefono Cellulare, Nero [Italia]",10.0
962436,ÐÐ½Ð´ÑÐµÐ¹,LG P920 Optimus 3D,10.0
76024,Amazon Customer,"OnePlus 3 (Graphite, 64 GB)",2.0
308465,Amazon Customer,"Asus Zenfone Max ZC550KL-6A076IN (Black, 3GB, ...",10.0
...,...,...,...
301402,Cliente Amazon,BQ Aquaris E5 HD - Smartphone libre Android (p...,2.0
143320,Giuseppe,"Samsung Galaxy A5 2016 Smartphone LTE, 16GB, Nero",10.0
113106,Frank,Sony Xperia XZ zwart / 32 GB,9.0
243649,e-bit,Smartphone Motorola Moto X 2Âª GeraÃ§Ã£o XT109...,6.0


## <span style="font-family: Arial; font-weight:bold;font-size:1.2em;color:#8B1A1A"> 3. Build a popularity based model and recommend top 5 mobile phones.

In [92]:
# Calculating score based upon Rating given by Authors

pop_grouped = df_sample.groupby('product').agg({'author': 'count'}).reset_index()
pop_grouped.rename(columns = {'author': 'score'},inplace=True)
pop_grouped.head()

,product,score
0,'Sony Xperia X (F5122) â White â Dual Sim ...,1
1,'Sony Xperia X (F5122) â rosa â Dual Sim (...,1
2,"(7.62 cm (3 )Afficheur/Ã©cran, 2 MPixCamÃ©ra;b...",1
3,"(CUBOT) GT88 5.5"" qHD 1.3GHz MTK6572 2-Core An...",1
4,(DG300 Versione Aggiornata)5'' DOOGEE VOYAGER2...,37


In [93]:
# Assigning Rank Based upon Rating Assigned by Authors

pop_sort = pop_grouped.sort_values(['score', 'product'], ascending = [0,1]) 
pop_sort['Rank'] = pop_sort['score'].rank(ascending=0, method='first')
popularity_recommendations = pop_sort.head()
popularity_recommendations

,product,score,Rank
20584,"Lenovo Vibe K4 Note (White,16GB)",4109,1.0
20583,"Lenovo Vibe K4 Note (Black, 16GB)",3451,2.0
29905,"OnePlus 3 (Graphite, 64 GB)",3212,3.0
29906,"OnePlus 3 (Soft Gold, 64 GB)",2798,4.0
14865,Huawei P8lite zwart / 16 GB,2121,5.0


In [51]:
# A function that will take a number for top mobiles and Provide Recommendations as Output
 
def recommend(top):     

    user_recommendations = popularity_recommendations[0:top] 
  
    return user_recommendations 

In [52]:
#Recommendations for top 5 mobile phones

recommend(5)

,product,score,Rank
20584,"Lenovo Vibe K4 Note (White,16GB)",4109,1.0
20583,"Lenovo Vibe K4 Note (Black, 16GB)",3451,2.0
29905,"OnePlus 3 (Graphite, 64 GB)",3212,3.0
29906,"OnePlus 3 (Soft Gold, 64 GB)",2798,4.0
14865,Huawei P8lite zwart / 16 GB,2121,5.0


## <span style="font-family: Arial; font-weight:bold;font-size:1.2em;color:#8B1A1A"> 4. Build a collaborative iltering model using SVD. You can use SVD from surprise or build it from scratch. Build a collaborative iltering model using **kNNWithMeans** from surprise. You can try both user-based and item-based model.
   (Note: Incase you’re building it from scratch you can limit your data points to 5000 samples if you face memory issues).  

In [98]:
# Limiting Dataset to 5k records 

df_sample = combined_phone_user_review_df.sample(n=1000000,random_state=612)

In [99]:
df_sample.shape

(1000000, 11)

In [100]:
df_sample.drop(df_sample.columns.difference(['author','product','score']),inplace=True,axis=1)

In [101]:
df_sample.shape

(1000000, 3)

In [65]:
df_sample

,author,product,score
510111,KHILESH KUMAR VERMA,"Lenovo Vibe K5 (Gold, VoLTE update)",10.0
104794,Evyta,Samsung Galaxy S6,8.0
1222473,VanRaZor,Sony Ericsson K810i,8.0
452910,ruga,Sony Xperia Z2 (Black),6.0
16933,einer Kundin,"Samsung Galaxy S7 edge Smartphone, 13,9 cm (5,...",10.0
...,...,...,...
98345,Bracrist,Samsung Galaxy S6 32GB (MetroPCS),10.0
1035917,luciano de santana,Smartphone Motorola Defy MB525,6.0
391663,Amazon Customer,"Samsung Galaxy C9 Pro (Gold, 6GB RAM)",6.0
951508,pnik87,HTC Desire HD,10.0


In [102]:
# Downsampling Dataset with top 50 users and top 50 products 
# sorted with the help of Ratings

min_ratings = 50
filter_products = df_sample['product'].value_counts() > min_ratings
filter_products = filter_products[filter_products].index.tolist()

min_user_ratings = 50
filter_users = df_sample['author'].value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

df_top = df_sample[(df_sample['product'].isin(filter_products)) & (df_sample['author'].isin(filter_users))]
print('The original data frame shape:\t{}'.format(df_sample.shape))
print('The new data frame shape:\t{}'.format(df_top.shape))

The original data frame shape:	(1000000, 3)
The new data frame shape:	(111595, 3)


In [108]:
# Loading Dataset for Surprise and
# assigning Rating on scale of 1-10

reader = Reader(rating_scale=(1,10))
data = Dataset.load_from_df(df_top[['author', 'product', 'score']], reader)

## <span style="font-family: Arial; font-weight:bold;font-size:1.2em;color:#8B1A1A"> 5.Evaluate the collaborative model. Print RMSE value.

In [109]:
# Applying k-fold cross validation with 5 folds to achieve better performance
 
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), KNNWithMeans(k=20)]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=5, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')  

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


,test_rmse,fit_time,test_time
Algorithm,,,
SVD,2.727998,1.204490,0.327243
KNNWithMeans,2.758859,0.501485,3.529213


In [139]:
trainset, testset = train_test_split(data, test_size=0.25)
algo_svd = SVD()
predictions_svd = algo_svd.fit(trainset).test(testset)
accuracy.rmse(predictions_svd)

RMSE: 2.7057


2.7057421214991217

In [122]:
test_author = 'ruga'
test_product = 'OnePlus 3 (Graphite, 64 GB)'

In [141]:
# Predicting score for random user and random product 

svd_test = algo_svd.predict(test_author,test_product)[3]
svd_test

8.571341065063999

In [142]:
# Applying Knn with Means with 30 means

trainset, testset = train_test_split(data, test_size=0.25)
algo_knn = KNNWithMeans(k=30)
predictions_knn = algo_knn.fit(trainset).test(testset)
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 2.7611


2.761055084184982

## <span style="font-family: Arial; font-weight:bold;font-size:1.2em;color:#8B1A1A"> 6. Predict score (average rating) for test users.

In [143]:
# Predicting score for random user and random product 

knn_test = algo_knn.predict(test_author,test_product)[3]

In [144]:
print('Predicted Score for Author',test_author,'and Product',test_product,'using SVD method is',format(np.round(svd_test,2) ))
print('Predicted Score for Author',test_author,'and Product',test_product,'using KNN with Means method is',format(np.round(knn_test,2)))

Predicted Score for Author ruga and Product OnePlus 3 (Graphite, 64 GB) using SVD method is 8.57
Predicted Score for Author ruga and Product OnePlus 3 (Graphite, 64 GB) using KNN with Means method is 7.8


## <span style="font-family: Arial; font-weight:bold;font-size:1.2em;color:#8B1A1A"> 7. Report your indings and inferences

**Observations and Inferences**

1. Both SVD and KNNWithMeans gives similar RMSE value but SVD gives better prediction score.

 ## <span style="font-family: Arial; font-weight:bold;font-size:1.2em;color:#8B1A1A"> 8. Try and recommend top 5 products for test users.

In [157]:
def get_top_n(predictions, n):
   

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [146]:
top_n = get_top_n(predictions_svd, n=5)

In [148]:
print('Top 5 recommendations for all test users are: \n')
for key,value in top_n.items(): print(key,'-> ',value,'\n')

Top 5 recommendations for all test users are: 

Cliente Amazon ->  [('Huawei P9 lite Vodafone, Nero [Italia]', 9.497203165637734), ('Huawei P9 lite Vodafone, Nero [Italia]', 9.497203165637734), ('Honor 7 Smartphone 4G, Display Full HD 5.2 Pollici, Processore Kirin 935 Octa Core 2.2 GHz, 16 GB Memoria Interna, 3 GB RAM, Fotocamera 20 MP, Argento', 9.473837166733796), ('Honor 7 Smartphone 4G, Display Full HD 5.2 Pollici, Processore Kirin 935 Octa Core 2.2 GHz, 16 GB Memoria Interna, 3 GB RAM, Fotocamera 20 MP, Argento', 9.473837166733796), ('Honor 7 Smartphone 4G, Display Full HD 5.2 Pollici, Processore Kirin 935 Octa Core 2.2 GHz, 16 GB Memoria Interna, 3 GB RAM, Fotocamera 20 MP, Argento', 9.473837166733796)] 

Amazon Customer ->  [('ZTE ZMax2 No Contract Phone - Carrier Packaging (AT&T)', 10), ('ZTE ZMax2 No Contract Phone - Carrier Packaging (AT&T)', 10), ('Doro Liberto 820 SIM-Free Smartphone - Pale Rose', 10), ('Huawei Honor 8 Unlocked Smartphone 32 GB Dual Camera - US Warranty (Sa

## <span style="font-family: Arial; font-weight:bold;font-size:1.2em;color:#8B1A1A"> 9. Try other techniques (Example: cross validation) to get better results.

In [152]:
%%time
svd_cv = cross_validate(SVD(),data, measures=['RMSE'], cv=5, verbose=False)
print('\n Mean svd cv score:', round(svd_cv['test_rmse'].mean(),2),'\n')
svd_cv


 Mean svd cv score: 2.73 

CPU times: user 7.73 s, sys: 23.3 ms, total: 7.75 s
Wall time: 7.77 s


{'test_rmse': array([2.72127429, 2.74221497, 2.73021053, 2.73316184, 2.70149481]),
 'fit_time': (1.1565654277801514,
  1.1338624954223633,
  1.1325464248657227,
  1.1477134227752686,
  1.1465885639190674),
 'test_time': (0.2655141353607178,
  0.21459722518920898,
  0.21940159797668457,
  0.44655418395996094,
  0.22911524772644043)}

In [155]:
%%time
knn_cv = cross_validate(KNNWithMeans(k=20),data, measures=['RMSE'], cv=5, verbose=False)
print('\n Mean knn_cv score:', round(knn_cv['test_rmse'].mean(),2),'\n')
knn_cv

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.

 Mean knn_cv score: 2.77 

CPU times: user 20.6 s, sys: 6.12 ms, total: 20.6 s
Wall time: 20.6 s


{'test_rmse': array([2.75036751, 2.78051735, 2.74495809, 2.77882413, 2.77567841]),
 'fit_time': (0.47345495223999023,
  0.5188567638397217,
  0.5124595165252686,
  0.5547475814819336,
  0.5357711315155029),
 'test_time': (3.3297135829925537,
  3.6662468910217285,
  3.4357542991638184,
  3.585078477859497,
  3.318143367767334)}

## <span style="font-family: Arial; font-weight:bold;font-size:1.2em;color:#8B1A1A"> 10. In what business scenario you should use popularity based Recommendation Systems ?

1. It is a type of recommendation system which works on the principle of popularity and or anything which is in trend. 

2. These systems check about the product or movie which are in trend or are most popular among the users and directly recommend those.

## <span style="font-family: Arial; font-weight:bold;font-size:1.2em;color:#8B1A1A"> 11. In what business scenario you should use CF based Recommendation Systems ?

You can use this technique to build recommenders that give suggestions to a user on the basis of the likes and dislikes of similar users.

## <span style="font-family: Arial; font-weight:bold;font-size:1.2em;color:#8B1A1A"> 12. What other possible methods can you think of which can further improve the recommendation for different users

1. We can use Hybrid recommendation techniques.

2. Social network-based recommendation techniques: Based on the user friends on social network.

3. Based on the amount of time spent on a particular item also, we can recommend items.